In [ ]:
import os
os.chdir("../")

In [ ]:
import torch
import torch.nn as nn
import json
import pickle

from src import CluBartModelForGenerativeQuestionAnswering
from utils import process_config
from datasets import load_dataset

from data import WikiTQWithReasonAsInputDataset

In [ ]:
with open("configs/wiki_tq_bootstrap/tapex.json", "rb") as f:
    config = json.load(f)
config = process_config(config)

In [ ]:
model = CluBartModelForGenerativeQuestionAnswering(config)
model.load_state_dict(torch.load("/dev/shm/tabllm/logs/table_question_answering_omnitab_cluster_with_flant5_reason_as_input_train_splitted/checkpoints/epoch=10.pt", map_location="cpu"))

In [ ]:
model.to("cuda:0")

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
train_dataset = WikiTQWithReasonAsInputDataset(dataset = dataset, config = config, data_type = "train")

In [ ]:
tokenizer = train_dataset.tokenizer

In [ ]:

import re
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """

    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
def evaluate(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = train_dataset.__getitem__(index)

    predicted_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"),
                                         max_new_tokens = 64, num_beams = 3, early_stopping=True).squeeze().cpu()

    predicted_sequence = tokenizer.decode(predicted_ids, skip_special_tokens=True)
    actual_sequence = tokenizer.decode(labels[labels != -100], skip_special_tokens=True)

    gold = [x.strip() for x in actual_sequence.split(",")]
    pred = [x.strip() for x in predicted_sequence.split(",")]

    pred = to_value_list(pred)
    gold = to_value_list(gold)

    verdict = check_denotation(gold, pred)

    return predicted_sequence, actual_sequence, verdict

In [ ]:
predicted_answer_list = []
actual_answer_list = []
verdict_list = []

In [ ]:
with open("datasets/bootstrap1_val_indices.pkl", "rb") as f:
    indices_list = pickle.load(f)

In [ ]:
for i in tqdm(range(len(indices_list)), position = 0, leave = True, total = len(indices_list)):

    idx = indices_list[i]
    predicted_sequence, actual_sequence, verdict = evaluate(idx)
    predicted_answer_list.append(predicted_sequence)
    actual_answer_list.append(actual_sequence)
    verdict_list.append(verdict)

In [ ]:
import numpy as np
(len(verdict_list) - np.sum(verdict_list)) / np.sum(verdict_list)

In [ ]:
predicted_answer_list

In [ ]:
actual_answer_list

In [ ]:
import torch.nn.functional as F

In [ ]:
def calculate_perplexity(index, i):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = train_dataset.__getitem__(index)


    predicted_answer_labels = tokenizer(answer = predicted_answer_list[i], add_special_tokens = config.tokenizer.add_special_tokens,
                            padding = config.tokenizer.padding, truncation = config.tokenizer.truncation, 
                            max_length = 64, return_tensors = config.tokenizer.return_tensors,
                            return_token_type_ids = config.tokenizer.return_token_type_ids,
                            return_attention_mask = config.tokenizer.return_attention_mask)["input_ids"]

    predicted_target_labels = predicted_answer_labels.clone().squeeze()
    predicted_target_labels[:-1] = predicted_target_labels[1:].clone()

    logits = model.model(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"),
                         decoder_input_ids = predicted_answer_labels.to("cuda:0")).logits.detach().cpu().squeeze()
    


    loss = F.cross_entropy(logits, predicted_target_labels, ignore_index=1)
    perplexity = torch.exp(loss)
    
    return perplexity

    token_probs = torch.nn.functional.softmax(logits, dim=-1)
    log_probs = torch.log2(token_probs)
    sum_log_probs = torch.sum(log_probs)

    perplexity = 2 ** (-sum_log_probs)


In [ ]:
perplexity_list = []

In [ ]:
for i, idx in tqdm(enumerate(indices_list), position = 0, leave = True, total = len(indices_list)):
    perplexity = calculate_perplexity(idx, i)
    perplexity_list.append(perplexity)

In [ ]:
perplexities = np.array(perplexity_list)
indices = np.array(indices_list)

verdicts = np.array(verdict_list)

In [ ]:
perplexities.shape

In [ ]:
verdicts.sum()

In [ ]:
perplexities[verdicts == True].mean()

In [ ]:
perplexities[verdicts == True].std()

In [ ]:
perplexities[verdicts == True][perplexities[verdicts == True] < perplexities[verdicts == True].mean()].shape

In [ ]:
perplexities[verdicts == True][perplexities[verdicts == True] < 1.0013].shape

In [ ]:
indices[verdicts == True][perplexities[verdicts == True] < 1.0013]

In [ ]:
train_dataset.reasons[10406]

In [ ]:
import pandas as pd

df = pd.read_csv("datasets/WikiTQReasoningData.csv")
df

In [ ]:
import pandas as pd

df = pd.read_csv("datasets/WikiTQReasoningDataBootstrap1.csv")

In [ ]:
idx = 850
question = df["question"][idx]
answers = df["answers"][idx]
table = df["table"][idx]
reason = df["reason"][idx]

print("Question: ", question, end = "\n\n")
print("Answers: ", answers, end = "\n\n")
print("Reason: ", reason, end = "\n\n")
print("Table: ", table, end = "\n\n")


In [ ]:
filtered_indices = indices[verdicts == True][perplexities[verdicts == True] < 1.0013]

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
new_dict = {"id": [],
            "question": [],
            "table": [],
            "answers": [],
            "cluster_label": [],
            "reason": []}

In [ ]:
for idx in tqdm(filtered_indices, position = 0, leave = True, total = len(filtered_indices)):

    if idx not in df.id.tolist():
        question = dataset["train"][int(idx)]["question"]
        table = dataset["train"][int(idx)]["table"]
        answers = dataset["train"][int(idx)]["answers"]
        cluster_label = -1
        reason = train_dataset.reasons[int(idx)]

        new_dict["id"].append(idx)
        new_dict["question"].append(question)
        new_dict["table"].append(table)
        new_dict["answers"].append(answers)
        new_dict["cluster_label"].append(cluster_label)
        new_dict["reason"].append(reason)

In [ ]:
new_df = pd.DataFrame(new_dict)

In [ ]:
df = df.drop(columns=["level_0"])

In [ ]:
df

In [ ]:
concat_df = pd.concat([df, new_df]).reset_index()

In [ ]:
concat_df

In [ ]:
with open("datasets/filtered_bootstrap1_train_indices.pkl", "wb") as f:
    pickle.dump(filtered_indices.tolist(), f)

In [ ]:
concat_df.to_csv("datasets/WikiTQReasoningDataBootstrap1.csv", index=False)